# Summary


This notebook demonstrates how to use a multivariate time series model in BigQuery ML to forecast PM2.5 values based on historical temperature and wind speed data from Seattle.

The process involves:
- Creating a dataset with daily PM2.5, temperature, and wind speed data for Seattle from the `bigquery-public-data.epa_historical_air_quality` dataset.
- Visualizing the dataset to understand the trends in PM2.5, temperature, and wind speed over time.
- Training an ARIMA_PLUS_XREG model to forecast PM2.5 values using temperature and wind speed as external regressors.
- Evaluating the model's ARIMA information and inspecting the coefficients.
- Forecasting PM2.5 values for 30 days after the training period.
- Evaluating the accuracy of the forecast.
- Explaining the forecast by retrieving components of the time series like seasonality, trend, and feature attributions.

# Initialize BigQuery Client

In [ ]:
from colabtools import auth,bigquery

scopes = [
    bigquery.SCOPES[0]
]

credentials = auth.get_user_oauth2_credentials(scopes)
project = 'bigquery-xreg' # @param {type:"string"}
bigquery.magics.context.project = project
bigquery.magics.context.credentials = credentials
from google.cloud.bigquery import magics
magics.context.credentials is credentials
bigquery_client = bigquery.Client(project=project, credentials=credentials)

# Create Dataset and Table

In [ ]:
create_dataset_query = """
CREATE SCHEMA IF NOT EXISTS `bqml_tutorial`;
"""

query_job = bigquery_client.query(create_dataset_query)
query_job.result()

print(f"Dataset created or already exists.")

In [ ]:
table_query = """
CREATE OR REPLACE TABLE `bqml_tutorial.seattle_air_quality_daily`
AS
WITH
  pm25_daily AS (
    SELECT
      avg(arithmetic_mean) AS pm25, date_local AS date
    FROM
      `bigquery-public-data.epa_historical_air_quality.pm25_nonfrm_daily_summary`
    WHERE
      city_name = 'Seattle'
      AND parameter_name = 'Acceptable PM2.5 AQI & Speciation Mass'
    GROUP BY date_local
  ),
  wind_speed_daily AS (
    SELECT
      avg(arithmetic_mean) AS wind_speed, date_local AS date
    FROM
      `bigquery-public-data.epa_historical_air_quality.wind_daily_summary`
    WHERE
      city_name = 'Seattle' AND parameter_name = 'Wind Speed - Resultant'
    GROUP BY date_local
  ),
  temperature_daily AS (
    SELECT
      avg(first_max_value) AS temperature, date_local AS date
    FROM
      `bigquery-public-data.epa_historical_air_quality.temperature_daily_summary`
    WHERE
      city_name = 'Seattle' AND parameter_name = 'Outdoor Temperature'
    GROUP BY date_local
  )
SELECT
  pm25_daily.date AS date, pm25, wind_speed, temperature
FROM pm25_daily
JOIN wind_speed_daily USING (date)
JOIN temperature_daily USING (date);
"""

results_df = bigquery_client.query(table_query).to_dataframe()

# Visualize Dataset

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import dates as mdates
import numpy as np

# Select data from the table and load it into results_df
query = """
SELECT *
FROM
  `bqml_tutorial.seattle_air_quality_daily`
WHERE
  date
  BETWEEN DATE('2012-01-01')
  AND DATE('2020-12-31') ORDER BY date
"""

job = bigquery_client.query(query)
results_df = job.to_dataframe()

# Convert 'date' column to datetime objects
results_df['date'] = pd.to_datetime(results_df['date'])
results_df = results_df.set_index('date')

# --- Reindex to handle missing dates ---
# Create a full date range from min to max date in the data
date_min = results_df.index.min()
date_max = results_df.index.max()
full_date_range = pd.date_range(start=date_min, end=date_max, freq='D')

# Reindex the DataFrame. Missing dates will be filled with NaN
results_df_reindexed = results_df.reindex(full_date_range)

# --- Plotting ---
plt.figure(figsize=(18, 7))

# Plotting from the reindexed DataFrame
plt.plot(results_df_reindexed.index, results_df_reindexed['pm25'], label='pm25', color='blue', linewidth=1.0)
plt.plot(results_df_reindexed.index, results_df_reindexed['temperature'], label='temperature', color='cyan', linewidth=1.0)
plt.plot(results_df_reindexed.index, results_df_reindexed['wind_speed'], label='wind_speed', color='magenta', linewidth=1.0)

plt.xlabel('Date', fontsize=12)
plt.ylabel('Values', fontsize=12)

# --- Y-axis customization ---
plt.ylim(0, 160)
plt.yticks(np.arange(0, 161, 20))

# --- X-axis customization ---
ax = plt.gca()

# Add a little padding to the x-axis limits
plt.xlim(date_min - pd.Timedelta(days=60), date_max + pd.Timedelta(days=60))

# AutoDateLocator helps pick 'nice' tick locations
locator = mdates.AutoDateLocator(minticks=12, maxticks=20)
# Formatter to match the '%b %d, %Y' style
formatter = mdates.DateFormatter('%b %d, %Y')

ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

# Rotate and align tick labels for readability
plt.setp(ax.get_xticklabels(), rotation=30, ha="right", fontsize=10)
plt.setp(ax.get_yticklabels(), fontsize=10)

# --- Grid ---
plt.grid(True, linestyle='-', alpha=0.6, color='#d3d3d3')

# --- Legend ---
plt.legend(loc='upper left', fontsize=11)

# Adjust layout to prevent labels from overlapping
plt.tight_layout()
plt.show()

# Train Model

In [ ]:
model_training_query = """
CREATE OR REPLACE
  MODEL
    `bqml_tutorial.seattle_pm25_xreg_model`
  OPTIONS (
    MODEL_TYPE = 'ARIMA_PLUS_XREG',
    time_series_timestamp_col = 'date',
    time_series_data_col = 'pm25')
AS
SELECT
  date,
  pm25,
  temperature,
  wind_speed
FROM
  `bqml_tutorial.seattle_air_quality_daily`
WHERE
  date
  BETWEEN DATE('2012-01-01')
  AND DATE('2020-12-31');
"""

query_job = bigquery_client.query(model_training_query)
query_job.result()

# Evaluate Model

In [ ]:
evaluation_query = """
SELECT
 *
FROM
 ML.ARIMA_EVALUATE(MODEL `bqml_tutorial.seattle_pm25_xreg_model`);
"""

evaluation_results = bigquery_client.query(evaluation_query).to_dataframe()
display(evaluation_results)

# Inspect Coefficients

In [ ]:
coefficients_query = """
SELECT
 *
FROM
 ML.ARIMA_COEFFICIENTS(MODEL `bqml_tutorial.seattle_pm25_xreg_model`);
"""

coefficients_results = bigquery_client.query(coefficients_query).to_dataframe()
display(coefficients_results)

# Forecast Data

In [ ]:
forecast_query = """
SELECT
  *
FROM
  ML.FORECAST(
    MODEL `bqml_tutorial.seattle_pm25_xreg_model`,
    STRUCT(30 AS horizon, 0.8 AS confidence_level),
    (
      SELECT
        date,
        temperature,
        wind_speed
      FROM
        `bqml_tutorial.seattle_air_quality_daily`
      WHERE
        date > DATE('2020-12-31')
    ));
"""

forecast_results = bigquery_client.query(forecast_query).to_dataframe()
display(forecast_results)

# Evaluate Forecast Accuracy

In [ ]:
evaluate_forecast_query = """
SELECT
  *
FROM
  ML.EVALUATE(
    MODEL `bqml_tutorial.seattle_pm25_xreg_model`,
    (
      SELECT
        date,
        pm25,
        temperature,
        wind_speed
      FROM
        `bqml_tutorial.seattle_air_quality_daily`
      WHERE
        date > DATE('2020-12-31')
    ),
    STRUCT(
      TRUE AS perform_aggregation,
      30 AS horizon));
"""

evaluate_forecast_results = bigquery_client.query(evaluate_forecast_query).to_dataframe()
display(evaluate_forecast_results)

# Explain Forecast

In [ ]:
explain_forecast_query = """
SELECT
  *
FROM
  ML.EXPLAIN_FORECAST(
    MODEL `bqml_tutorial.seattle_pm25_xreg_model`,
    STRUCT(30 AS horizon, 0.8 AS confidence_level),
    (
      SELECT
        date,
        temperature,
        wind_speed
      FROM
        `bqml_tutorial.seattle_air_quality_daily`
      WHERE
        date > DATE('2020-12-31')
    ));
"""

explain_forecast_results = bigquery_client.query(explain_forecast_query).to_dataframe()
display(explain_forecast_results)